# ALI numerical algorithm for two-stream transfer

## Imports

In [ ]:
import numpy as np

## 1) Set parameters and boundary values

### Set the grid

In [ ]:
# Set the number of grid points
n = 5

# Set the 1D grid
z = np.linspace(0.,1.,num=n)

# Set delta z (grid spacing value)
del_z = z[1]

### Gas variables

In [ ]:
# Set the thermal source function (aka the Planck function)
# In this example it is constant at each depth
B = np.ones(n)

# Photon destruction probability
# In this example it is constant 
epsilon = np.ones(n)*0.1

In [ ]:
# Set the extinction coefficient
def alpha(z):
    """
    Input: Depth z 
    Output: Extinction coefficient at the depth value given
    """
    
    return(10.**(5. - 6.*z)) 

# For the example problem this is 0
# but it will not always be 0

In [ ]:
alpha_e = alpha(z)

In [ ]:
# Optical depth, (delta tau)
# Right now it's set based on example 4.4.5
# but there will be more options to 
# calculate this depending on what 
# information is given in the problem
# It is also constant here because our
# grid spacing is equal
def tau_fn(n,delta_z):
    """
    Input:
        n = number of grid points
        alpha_e = array of extinction coefficients for the grid
    Output:
        tau = array of the optical depth at each grid point
    """
    
    tau = np.zeros((n-1))
    for i in range(0,n-1):
        j = i + delta_z/2.
        tau[i] = -np.sqrt(3.)*alpha_e(j)*delta_z
    return(tau)

In [ ]:
tau = tau_fn(n,alpha_e)

### Set boundary and initial values

In [ ]:
# Set the initial values for I
# 0 everywhere except at the top boundary
I_plus = np.zeros((n))
I_plus[0] = 1.
I_minus = I_plus

# Calculate J from the two rays
# The mean specific intensity
J = 0.5*(I_plus + I_minus)

## 2) Create interpolation functions and a matrix solver

### Quadratic Bezier Interpolation for I(U)

In [ ]:
# Interpolation coefficients 
# Only need interpolation if working in 2D or 3D
# Still not exactly sure how to define t
#t = 0.5
#m = 1. - t

#def I_interp(i,j):
#    """
#    Input: location on grid
#        i = x-coordinate 
#        j = y-coordinate
#    Output: Value of thrid-order-quadrature specific intensity value 
#        at the input location on grid
#    """
#    return(I[i-1,j]*u**2 + I[i+1,j]*2.*u*t + I[i+2,j]*t**2)

### Third order quadratic interpolation for the source contribution

In [ ]:
# Interpolation coefficients
e0 = np.ones((n-1))
e1 = tau - e0
e2 = tau**2 - (2.*e1)
u = e0 + (e2 - 3.*tau*e1)/(2.*tau**2)
p = ((2.*tau*e1)-e2)/(tau**2)
d = (e2 - tau*e1)/(2.*tau**2)

# Define the 1D source term interpolation function
def S_interp(i,S,plus=True):
    """
    Input: 
        i = location on grid
        S = Source value at each gridpoint
        plus = True if we are dealing with the ray that
               is being integrated from the bottom to top
               False if integrated from the top to bottom
    Output: Value of thrid-order-quadrature source value
            for the two-stream approximation
        at the input location on grid
    """
    
    if plus==True:
        return(u[i]*S[i-1] + p[i]*S[i] + d[i]*S[i+1])
    else:
        return(u[i]*S[i+1] + p[i]*S[i] + d[i]*S[i-1])

In [ ]:
# Interpolate to find I based on the
# initial S and above coefficients
for i in range(1,n-2):
    # Integrate from the bottom to top (one ray)
    I_plus[i] = np.exp(-tau[i-1])*I_plus[i-1] + S_interp(i,S,plus=True)
    # Integrate from the top to bottom (second ray)
    I_minus[i] = np.exp(-tau[i])*I_minus[i+1] + S_interp(i,S,plus=False)

### Partial Lambda Operator

In [ ]:
# These are the lower diagonal,
# diagonal, and upper diagonal
# pieces of the simplified lambda
# operator. These will vary across 
# the grid if tau varies with z

# Lower Diagonal 
lmbda_l = 0.5*(u+d)

# Diagonal 
lmbda_d = 0.5*(p+p)

# Upper Diagonal
lmbda_u = 0.5*(d+u)

# Construct the operator
lmbda = np.zeros(n,n)
for i in range(0,n-1):
    lmbda[i,i] = lmbda_d[i]
    if i==0:
        lmbda[i,i+1] = lmbda_u[i]
    elif i==(n-1):
        lmbda[i,i-1] = lmbda_l[i]
    else:
        lmbda[i,i+1] = lmbda_u[i]
        lmbda[i,i-1] = lmbda_l[i]

## M matrix

In [ ]:
# M* = [Identity - (1-epsilon)*Lambda*]
# Where Lambda* is the partial lambda operator 
# The matrix equation that will be solved is:
# M*S = epsilon*B
M = np.ones(n,n) - lmbda*(np.ones(n)-epsilon)

### Forward elimination and backward substitution subroutine

In [ ]:
# Define the functions for the coefficients
# used in the Thomas Algorithm which is used 
# and explained below
def gamma_beta_fn(mat, y, n):
    """
    Input:
        M = the tridiagonal matrix to be sovled (nxn)
        y = the solution array in the matrix equation: Mx = y (nx1)
        n = the number of rows in matrix m
    Output:
        g = the array of gamma constants (nx1)
        b = the array of beta constants (nx1)
    """
    
    # Define column arrays for necessary row-specific constants
    # This automatically sets gamma[0] = beta[0] = 0 which is 
    # necessary for the problem
    g = np.zeros((n))
    b = np.zeros((n))
    
    for i in range(0,n-2):
        if i==0:
            g[i+1] = ((-mat[i,i+1])/mat[i,i])
            b[i+1] = y[i]/mat[i,i]
        else:
            g[i+1] = ((-mat[i,i+1])/(g[i]*mat[i,i-1] + mat[i,i])) 
            b[i+1] = (y[i] - b[i]*mat[i,i-1])/mat[i,i]
    
    return(g,b)

In [ ]:
# Thomas Algorithm
# Forward elimination for a tridiagonal matrix
# Based on the algorithm found in Dullemond Ch4 notes
# Backward Substition is then applied
def tri_solver(M,S,epsilon,B,del_z):
    """
    Input: 
        J = specific intensity at every grid point
        alpha_t = total opacity
        alpha_a = absorption opacity
        B = thermal source function
    Output:
        A new matrix or vector of J values 
    """
    
    #-----------------------------------------------------------
    # Perform forward elimination and backward substitution on M
    # Thomas Algorithm: a simplified form of Gaussian Elimination
    # specifically for tridiagonal matrices 
    
    # Calculate Y = alpha_a(J - B) from MJ = alpha_a(J - B) = Y
    Y = epsilon*B
    # Calculate the necessary gamma and beta constants
    # This is essentially storing information obtained from 
    # forward substitution 
    gamma, beta = gamma_beta_fn(M, Y, n)
    # We use these constants to complete the calculation for 
    # what is essentially a back substitution to finish the
    # calculation of MX = Y
    X = np.zeros((n+1))
    X[0:n] = S
    for k in range(1,n):
        # Actually need to go from n, n-1, n-2,...,1
        # So create a new place holder that goes backwards
        c = (n) - k
        X[c-1] = gamma[c]*X[c] + beta[c]
    
    # Return the new solution to J
    return(X[0:n])

## 3) Solve for converged S

In [ ]:
# Solve for S -- I don't think this will be here...
# ----------------------------------------------------------------------
# Solve for the source function via a formal solution
#S = epsilon*B + (np.ones(n) - epsilon)*J

# Solve for I
# ----------------------------------------------------------------------
# For 2D and 3D use the function defined for 
# quadratic Bezier interpolation to find the new value for I(U)
# For 1D just use the i-1 grid point for the exponential term
# Use a different third-order-quadrature method for S contributions
#for i in range(0,n-1):
#    I[i] = np.exp(-tau)*I[i-1] + S_interp(i)

# Solve for J
# ----------------------------------------------------------------------
# The 1D diffusion equation is a 2nd order PDE where J is the solution
# The numerical representation used for this PDE is Central Space
# Forward substitution + backward substitution are used to solve the PDE